# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import os
import numpy as np
import pandas as pd
import pkg_resources
import joblib
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from matplotlib import pyplot as plt
from sklearn import datasets
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Workspace, ScriptRunConfig, Environment

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-268555
aml-quickstarts-268555
westeurope
aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee


In [3]:
experiment_name = 'CapstoneExp-Hyper'
project_folder = './Capstone-project'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
CapstoneExp-Hyper,quick-starts-ws-268555,Link to Azure Machine Learning studio,Link to Documentation


In [4]:
found = False
key = "capstone-dataset"
description_text = "The UCI Machine Learning Repository - Predict Students' Dropout and Academic success"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
    
if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://github.com/duclinh/Capstone-Azure-Machine-Learning/blob/main/student-dropout-and-academeic-success-dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

df = dataset.to_pandas_dataframe()
df.describe() 

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,Marital_status,Application_mode,Application_order,Course,Daytime_attendance,Previous_qualification,Previous_qualification_grade,Nacionality,Mother_qualification,Father_qualification,...,Curricularunits1stsem_withoutevaluations,Curricularunits2ndsem_credited,Curricularunits2ndsem_enrolled,Curricularunits2ndsem_evaluations,Curricularunits2ndsem_approved,Curricularunits2ndsem_grade,Curricularunits2ndsem_withoutevaluations,Unemploymentrate,Inflationrate,GDP
count,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,...,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000,4424.000000
mean,1.178571,18.669078,1.727848,8856.642631,0.890823,4.577758,132.613314,1.873192,19.561935,22.275316,...,0.137658,0.541817,6.232143,8.063291,4.435805,10.230206,0.150316,11.566139,1.228029,0.001969
std,0.605747,17.484682,1.313793,2063.566416,0.311897,10.216592,13.188332,6.914514,15.603186,15.343108,...,0.690880,1.918546,2.195951,3.947951,3.014764,5.210808,0.753774,2.663850,1.382711,2.269935
min,1.000000,1.000000,0.000000,33.000000,0.000000,1.000000,95.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.600000,-0.800000,-4.060000
25%,1.000000,1.000000,1.000000,9085.000000,1.000000,1.000000,125.000000,1.000000,2.000000,3.000000,...,0.000000,0.000000,5.000000,6.000000,2.000000,10.750000,0.000000,9.400000,0.300000,-1.700000
50%,1.000000,17.000000,1.000000,9238.000000,1.000000,1.000000,133.100000,1.000000,19.000000,19.000000,...,0.000000,0.000000,6.000000,8.000000,5.000000,12.200000,0.000000,11.100000,1.400000,0.320000
75%,1.000000,39.000000,2.000000,9556.000000,1.000000,1.000000,140.000000,1.000000,37.000000,37.000000,...,0.000000,0.000000,7.000000,10.000000,6.000000,13.333333,0.000000,13.900000,2.600000,1.790000
max,6.000000,57.000000,9.000000,9991.000000,1.000000,43.000000,190.000000,109.000000,44.000000,44.000000,...,12.000000,19.000000,23.000000,33.000000,20.000000,18.571429,12.000000,16.200000,3.700000,3.510000


In [5]:
amlcompute_cluster_name = "capstone-auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2',
                                                           max_nodes=5)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [30]:
from azure.ai.ml.sweep import BanditPolicy

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.1, delay_evaluation = 5, evaluation_interval = 1)

#TODO: Create the different params that you will be using during training
param_sampling =  RandomParameterSampling(
    {
        '--batch-size': choice(25, 50, 100),
        '--first-layer-neurons': choice(10, 50, 200, 300, 500),
        '--second-layer-neurons': choice(10, 50, 200, 500),
        '--learning-rate': loguniform(-6, -1)
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")
    
# Setup environment for your training run
env = Environment.get(workspace=ws, name="AzureML-Minimal")

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
                        source_directory="./",
                        compute_target=compute_target,
                        script="train.py",
                        environment=env
                    )
#TODO: Create your estimator and hyperdrive config
hyperdrive_run_config = HyperDriveConfig(
                                    run_config=src,
                                    hyperparameter_sampling=param_sampling,
                                    policy=early_termination_policy,
                                    primary_metric_name="validation_acc",
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=100,
                                    max_concurrent_runs=4
                                )

NameError: name 'loguniform' is not defined

In [25]:
#TODO: Submit your experiment
hyperDrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [26]:
RunDetails(hyperDrive_run).show()
for children_run in hyperDrive_run.get_children():
    print('------')
    print(children_run)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [27]:
hyperDrive_run.wait_for_completion(show_output=True)

RunId: HD_4e6f6552-6056-43b0-807e-fcfd6eb48241
Web View: https://ml.azure.com/runs/HD_4e6f6552-6056-43b0-807e-fcfd6eb48241?wsid=/subscriptions/aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee/resourcegroups/aml-quickstarts-268555/workspaces/quick-starts-ws-268555&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-10-11T20:15:16.686567][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2024-10-11T20:15:17.2226908Z][SCHEDULER][INFO]Scheduling job, id='HD_4e6f6552-6056-43b0-807e-fcfd6eb48241_0' 
[2024-10-11T20:15:17.3262008Z][SCHEDULER][INFO]Scheduling job, id='HD_4e6f6552-6056-43b0-807e-fcfd6eb48241_1' 
[2024-10-11T20:15:17.4468688Z][SCHEDULER][INFO]Scheduling job, id='HD_4e6f6552-6056-43b0-807e-fcfd6eb48241_2' 
[2024-10-11T20:15:17.5626478Z][SCHEDULER][INFO]Scheduling job, id='HD_4e6f6552-6056-43b0-807e-fcfd6eb48241_3' 
[2024-10-11T20:15:17.525418][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Execution failed. User process '/azureml-envs/azureml_7311f932e907bdb37346f513c1147dc7/bin/python' exited with status code 1. Please check log file 'user_logs/std_log.txt' for error details. Error: Traceback (most recent call last):\n  File \"<string>\", line 197, in <module>\n  File \"<string>\", line 193, in main\n  File \"/azureml-envs/azureml_7311f932e907bdb37346f513c1147dc7/lib/python3.6/runpy.py\", line 261, in run_path\n    code, fname = _get_code_from_file(run_name, path_name)\n  File \"/azureml-envs/azureml_7311f932e907bdb37346f513c1147dc7/lib/python3.6/runpy.py\", line 231, in _get_code_from_file\n    with open(fname, \"rb\") as f:\nFileNotFoundError: [Errno 2] No such file or directory: 'train.py'\n\n Marking the experiment as failed because initial child jobs have failed due to user error",
        "messageParameters": {},
        "d

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperDrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Regularization Strength:',best_run_metrics['Regularizat

In [ ]:
#TODO: Save the best model
best_run.register_model(model_name = "hyperdrive_best_run.pkl", model_path = './outputs/')

print(best_run)
best_run.download_file( name= './outputs/hyper-model.pkl')

In [ ]:
# List registered models to verify if model has been saved
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

